<a href="https://colab.research.google.com/github/AvokrichA/Apache_Spark/blob/main/Home_Task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [ ]:
!pip install pyspark >> None

In [20]:
!python -m spylon_kernel install

/usr/bin/python3: No module named spylon_kernel


In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.types import DoubleType

In [33]:
spark = SparkSession.builder.appName("Home_Task_5").getOrCreate()


**Используя Spark прочитайте данные из файла csv**

In [46]:
df = spark.read.option("header",True).csv("gdrive/My Drive/Books_HT5_3.csv")
df.show()

+--------------------+--------------------+--------------------+-----+------+
|               title|              author|               genre|sales|  year|
+--------------------+--------------------+--------------------+-----+------+
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|
|"""The Lord of th...|  ""J.R.R. Tolkien""|         ""Fantasy""| 3000| 1954"|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|
|"""The Catcher in...|   ""J.D. Salinger""|           ""Novel""| 2000| 1951"|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|
+--------------------+--------------------+--------------------+-----+------+



**Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров**

In [48]:
df_filter = df.filter(df.sales > 3000)
df_filter.show()

+--------------------+--------------------+--------------------+-----+------+
|               title|              author|               genre|sales|  year|
+--------------------+--------------------+--------------------+-----+------+
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|
+--------------------+--------------------+--------------------+-----+------+



**Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.**

In [56]:
window_spec = Window.partitionBy("genre")
sale_genre = func.sum("sales").over(window_spec)
df_sale_genre = df.withColumn("sale_genre", sale_genre)
df_sale_genre.show()

+--------------------+--------------------+--------------------+-----+------+----------+
|               title|              author|               genre|sales|  year|sale_genre|
+--------------------+--------------------+--------------------+-----+------+----------+
|"""The Lord of th...|  ""J.R.R. Tolkien""|         ""Fantasy""| 3000| 1954"|    3000.0|
|"""The Catcher in...|   ""J.D. Salinger""|           ""Novel""| 2000| 1951"|    6500.0|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|    6500.0|
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|    5000.0|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|    4000.0|
+--------------------+--------------------+--------------------+-----+------+----------+



**Отсортируйте данные по общему объему продаж в порядке убывания**

In [62]:
df_sale_genre.select(["genre", "sale_genre"]).distinct().orderBy(df_sale_genre.sale_genre.desc()).show()

+--------------------+----------+
|               genre|sale_genre|
+--------------------+----------+
|           ""Novel""|    6500.0|
| ""Science Fiction""|    5000.0|
| ""Southern Gothic""|    4000.0|
|         ""Fantasy""|    3000.0|
+--------------------+----------+

